# Importing the raw data

In [18]:
import pandas as pd
import csv
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
#

Importing from the local drive

In [2]:
#file = 'CSVs/dummy.csv'
file = 'CSVs/data_full_04-18.csv'

In [3]:
rawData = pd.read_csv(file)
#rawData

In [19]:
pd.options.display.max_colwidth = 100000
sentencesdf = rawData[['site', 'title', 'date','writer', 'content', 'comments', 'label']].copy()

#sorting out the rows with empty 'content' columns
sentencesdf = sentencesdf[sentencesdf['content'].notna()]
#print(sentencesdf['content'][2])
sentencesdf.reset_index(inplace=True, drop=True)

#Splitting the articles into sentences
for i in range(len(sentencesdf['content'])):
    sentencesdf['content'][i] = sentencesdf['content'][i].split('|')
    #print(sentencesdf['content'][i])
#sentencesdf

C:\Users\simon\AppData\Local\Temp/ipykernel_12720/2195900630.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentencesdf['content'][i] = sentencesdf['content'][i].split('|')


In [14]:
sentencesdf.reset_index(inplace=True, drop=True)
#sentencesdf['content']

In [15]:
#Addig back the properties, that are constant throughout the article
rows = []
_ = sentencesdf.apply(lambda row: [rows.append([row['site'], row['title'], row['date'], row['writer'], nn, row['comments'], row['label']])
                         for nn in row.content], axis=1)

#rows

In [7]:
basedf = pd.DataFrame(rows, columns=sentencesdf.columns).set_index(['title'])
#print(basedf)
basedf['content'].replace('', np.nan, inplace=True)
basedf.dropna(subset=['content'], inplace=True)

In [8]:
basedf = basedf.reset_index()
#basedf

In [16]:
#Adding new column for sentence labels
basedf['sentencelabel'] = 0

#Exporting the sentences for manual labeling
labelingdf = basedf[['content', 'sentencelabel']]
#print(labelingdf)

from datetime import datetime
labelingdf.to_csv('data_for_labeling' + datetime.today().strftime('%Y-%m-%d') + '.csv', index=False)

#basedf

In [35]:
labelingdf['content']
one_hot("Mi történt a választáson?", 1000)
vocab_size = 1000
encoded_reviews = [one_hot(d, vocab_size) for d in labelingdf['content']]
#print(encoded_reviews)

max_length = 30
padded_reviews = pad_sequences(encoded_reviews, maxlen=max_length, padding='post')
print(padded_reviews)

embeded_vector_size = 20

model = Sequential()
model.add(Embedding(vocab_size, embeded_vector_size, input_length=max_length,name="embedding"))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

X_embedding = padded_reviews
y_embedding = labelingdf['sentencelabel']

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())

model.fit(X_embedding, y_embedding, epochs=50, verbose=0)

loss, accuracy = model.evaluate(X_embedding, y_embedding)
accuracy

weights = model.get_layer('embedding').get_weights()[0]
len(weights)

weights[3]

[[398 568  97 ...   0   0   0]
 [ 97 727  97 ...   0   0   0]
 [361 486  64 ...   0   0   0]
 ...
 [400 710   0 ...   0   0   0]
 [ 97 537 588 ...   0   0   0]
 [208 808   0 ...   0   0   0]]
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 30, 20)            20000     
_________________________________________________________________
flatten_8 (Flatten)          (None, 600)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 601       
Total params: 20,601
Trainable params: 20,601
Non-trainable params: 0
_________________________________________________________________
None
43/43 [==============================] - 0s 785us/step - loss: 6.4487e-05 - accuracy: 1.0000


array([ 0.11100065, -0.0981008 , -0.0817989 ,  0.10216606,  0.03926456,
       -0.07253637,  0.10962544,  0.12481873,  0.08125506, -0.11500516,
        0.04533995, -0.05210059, -0.00807217,  0.07939596,  0.06391914,
        0.04738462,  0.06991066,  0.05929901, -0.01178173, -0.04102528],
      dtype=float32)

In [10]:
file = './data_for_labeling2022-04-19.csv'
sentencelabels = pd.read_csv(file, index_col=False)
basedf['sentencelabel'] = sentencelabels['sentencelabel']
#sentencelabels
#basedf

In [11]:
from datetime import datetime
basedf.to_csv('data_for_training' + datetime.today().strftime('%Y-%m-%d') + '.csv', index=False)

In [ ]:
#basedf